# Ceneo Scraper

## Bilioteki

In [18]:
import os
import json
import requests
from bs4 import BeautifulSoup

In [19]:
def extract(ancestor, selector=None, attribute=None, returns_list=False):
    if selector:
        if returns_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)],
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()


In [20]:
selectors = {
    "opinion_id": (None, "data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recommendation": ("span.user-post__author-recomendation > em",),
    "rating": ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "pros": ("div.review-feature__title--positives~div.review-feature__item",None,True),
    "cons": ("div.review-feature__title--negatives~div.review-feature__item",None,True),
    "useful": ('span[id^="votes-yes"]',),
    "useless" : ('span[id^="votes-no"]',),
    "post_date" : ('span.user-post__published > time:nth-child(1)',"datetime"),
    "purchase_date" : ('span.user-post__published > time:nth-child(2)',"datetime"),
}

## wysyłanie żądania do zasobu do serwera

In [21]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-1"

In [22]:
all_opinions = []
while url:
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text,"html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = f"https://www.ceneo.pl"+ extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = None

TypeError: 'NoneType' object is not subscriptable

In [ ]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json","w",encoding="UTF-8") as jf: 
    json.dump(all_opinions, jf , indent=4, ensure_ascii=False)

In [ ]:
len(all_opinions)

## Ekstrakcja składowych pojedynczej opinii

|Składowa|Selektor|zmienna|
|--------|--------|--------|
|id opinii|["js_product-review"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|user-post__score.count|rating|
|treść|user-post__text|content|
|lista zalet|review-feature__title--positives~div.review-feature__item|pros|
|lista wad|review-feature__title--negatives~div.review-feature__item|cons|
|dla ilu przydatna|span[id^="votes-yes"]|useful|
|dla ilu nieprzydatna|span[id^="votes-no"]|useless|
|data wystawienia|span.user-post__published > time:nth-child(1)|post_date|
|data zakupu|span.user-post__published > time:nth-child(2)|purchase_date|

In [ ]:
print(json.dumps(single_opinion, indent=4, ensure_ascii=False))
print(json.dumps(all_opinions, indent=4, ensure_ascii=False))

## zadanie 1